In [272]:
from EAExperiment import EAExperiment
import pandas as pd
import functools
import numpy as np
from metrics import evaluate
import math

from sklearn.model_selection import RepeatedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [275]:
uniform(loc=2, scale=15)

In [286]:
# Decission tree
#distributions_dt = dict(max_depth=list(range(2,16)))
regrt_model = DecisionTreeRegressor(max_depth=5)

# AdaBoost with Decission tree
#distributions_ada = {'estimator__base_estimator__max_depth':list(range(2,16))}
ada_regrt_model = MultiOutputRegressor(AdaBoostRegressor(DecisionTreeRegressor(),
                          n_estimators=300), n_jobs=-1)

ada_regrt_chain_model = RegressorChain(AdaBoostRegressor(DecisionTreeRegressor(max_depth=5),
                          n_estimators=300))

# Gradient boosting
grboost_model = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=300,loss='ls', learning_rate=0.1,
                                                               max_depth=5), n_jobs=-1)
grboost_chain_model = RegressorChain(GradientBoostingRegressor(n_estimators=300,loss='ls', learning_rate=0.1,
                                                               max_depth=5))
# Gaussian process
kernel = DotProduct() + WhiteKernel()
gausspr_model = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel), n_jobs=-1)
gausspr_chain_model = RegressorChain(GaussianProcessRegressor(kernel=kernel))

#Random forest
rand_forest_model = RandomForestRegressor(n_estimators=300,max_depth=5)

models = {
    "decision_tree_regressor": regrt_model,
    "adaboost_tree_regressor": ada_regrt_model,
    "adaboost_tree_regressor_chain":ada_regrt_chain_model,
    "gaussian_process_regressor": gausspr_model,
    "gaussian_process_regressor_chain":gausspr_chain_model,
    "gradient_boost": grboost_model,
    "gaussian_process_regressor_chain":grboost_chain_model,
    "random_forest": rand_forest_model
}

In [287]:
def create_propositional_table_dataframe(df,w, target, h):
    columns =[]
    for i in range(w,0,-1):
        columns.extend([s + "_lag"+str(i-1) for s in df.columns])
    for t in target:
        for j in range(h):
            columns.append(t+"_ahead"+str(j+1))
    dataframe = pd.DataFrame(columns=columns)
    return dataframe
    
def create_propositional_table(df, w, h, target):
    columns =[]
    for i in range(w,0,-1):
        columns.extend([s + "_lag"+str(i-1) for s in df.columns])
    for t in target:
        for j in range(h):
            columns.append(t+"_ahead"+str(j+1))
    dataframe = pd.DataFrame(columns=columns)
    
    indexes = []
    for i in range((len(df)-w-h+1)):
        window = df.iloc[i:(i+w)]
        row = window.values.reshape(1, len(window.columns)*len(window))
        targets = {}
        for t in target:
            row = np.append(row, df[t].iloc[(i+w):(i+w+h)])
            
        dataframe.loc[i]=row.reshape(1, len(row))[0]
        indexes.append(window.index[-1])
    
    dataframe = dataframe.set_index(pd.Series(indexes))
    
    return dataframe

In [288]:
def calculate_metrics(y_test, y_pred, target, model_name, metrics = ('mae', 'mape', 'rmse','mse')):
    horizons = y_test.columns.values
    index_horizons = np.append(horizons,target+"_mean")
    index = [np.array([model_name for i in range(7)]), index_horizons]
    metrics = pd.DataFrame(evaluate(y_test, y_pred, metrics=metrics))
    metrics.loc[6] = metrics.values.mean(axis=0)
    metrics = metrics.set_index(index)
    
    return metrics

In [289]:
def execute_baseline(df_propositional_table, models, target, train_split=0.75):
    tran_split = int(df_propositional_table.shape[0]*0.75)

    X_train = df_propositional_table.iloc[:tran_split, :-6]
    y_train = df_propositional_table.iloc[:tran_split, -6:]

    X_test = df_propositional_table.iloc[tran_split:, :-6]
    y_test = df_propositional_table.iloc[tran_split:, -6:]
    
    test_metrics_global = None
    train_metrics_global = None
    
    results_global = None
    for name, model in models.items():
        
        best_model = None
        last_mean_mae = float('inf')
        print("Training "+name+"....")
        rkf = RepeatedKFold(n_splits=2, n_repeats=5)
        for train_index, test_index in rkf.split(X_train):
            X_train_fold, X_test_fold = X_train.iloc[train_index,:], X_train.iloc[test_index,:]
            y_train_fold, y_test_fold = y_train.iloc[train_index,:], y_train.iloc[test_index,:]
            model.fit(X_train_fold,y_train_fold)
            mean_mae = calculate_metrics(y_test_fold, model.predict(X_test_fold), target, name)["mae"][6]
            if mean_mae < last_mean_mae:
                best_model = model
        
        test_pred = best_model.predict(X_test)
        train_pred = best_model.predict(X_train)
        
        test_metrics = calculate_metrics(y_test, test_pred, target, name)
        train_metrics = calculate_metrics(y_train, train_pred, target, name)
        
        if test_metrics_global is None and train_metrics_global is None:
            test_metrics_global = test_metrics
            train_metrics_global = train_metrics
        else:
            test_metrics_global = test_metrics_global.append(test_metrics)
            train_metrics_global = train_metrics_global.append(train_metrics)
        
        test_pred = pd.DataFrame(test_pred, columns=y_test.columns, index=y_test.index)
        results_model = pd.concat({"Real": y_test, "Pred": test_pred}, axis=1, names=["Type", "Horizon"])
        results = pd.concat({name: results_model}, axis=1, names=["Model", "Type", "Horizon"])
        if results_global is None:
            results_global = results
        else:
            results_global = pd.concat([results_global, results], axis=1, join='inner')
            
        
    return train_metrics_global, test_metrics_global, results_global
            

In [290]:
def execute(df, window, horizon, targets, models, train_split=0.75):
    for target in targets:
        print("============== "+target+" ==============")
        df_propositional = create_propositional_table(df, 12, 6, [target])
        train_metrics, test_metrics, results = execute_baseline(df_propositional, models, target, train_split)
        train_metrics.to_pickle("metrics/train_metrics_"+target)
        test_metrics.to_pickle("metrics/test_metrics_"+target)
        results.to_pickle("results/results_"+target)

In [291]:
df = pd.read_csv('data.csv')
df = df.set_index("PERIODO")

In [292]:
%%time
execute(df, 12, 6, ["PESO_7", "PESO_8", "PESO_20"], models)

============== PESO_7 ==============
Training decision_tree_regressor....
Training adaboost_tree_regressor....
Training adaboost_tree_regressor_chain....
Training gaussian_process_regressor....
Training gaussian_process_regressor_chain....
Training gradient_boost....
Training random_forest....
============== PESO_8 ==============
Training decision_tree_regressor....
Training adaboost_tree_regressor....
Training adaboost_tree_regressor_chain....
Training gaussian_process_regressor....
Training gaussian_process_regressor_chain....
Training gradient_boost....
Training random_forest....
============== PESO_20 ==============
Training decision_tree_regressor....
Training adaboost_tree_regressor....
Training adaboost_tree_regressor_chain....
Training gaussian_process_regressor....
Training gaussian_process_regressor_chain....
Training gradient_boost....
Training random_forest....
CPU times: user 3min 48s, sys: 777 ms, total: 3min 48s
Wall time: 4min 25s


In [296]:
results_p7 = pd.read_pickle("results/results_PESO_8")

In [307]:
metrics_p7 = pd.read_pickle("metrics/test_metrics_PESO_20")

In [308]:
metrics_p7

mae      mape  \
decision_tree_regressor          PESO_20_ahead1  30962.484848  0.508277   
                                 PESO_20_ahead2  23725.090909  0.375548   
                                 PESO_20_ahead3  39487.136364  0.443219   
                                 PESO_20_ahead4  25719.000000  0.311406   
                                 PESO_20_ahead5  54175.833333  0.700960   
                                 PESO_20_ahead6  18252.000000  0.279728   
                                 PESO_20_mean    32053.590909  0.436523   
adaboost_tree_regressor          PESO_20_ahead1  25949.727273  0.399326   
                                 PESO_20_ahead2  25742.818182  0.448474   
                                 PESO_20_ahead3  21735.363636  0.251215   
                                 PESO_20_ahead4  20122.454545  0.218948   
                                 PESO_20_ahead5  18249.181818  0.256245   
                                 PESO_20_ahead6  21433.818182  0.311748   
                                 PESO_20_mean    22205.560606  0.314326   
adaboost_tree_regressor_chain    PESO_20_ahead1  27878.545455  0.394543   
                                 PESO_20_ahead2  25273.272727  0.397730   
                                 PESO_20_ahead3  18065.727273  0.232092   
                                 PESO_20_ahead4  22316.727273  0.263077   
                                 PESO_20_ahead5  15676.227273  0.205874   
                                 PESO_20_ahead6  17300.290909  0.271183   
                                 PESO_20_mean    21085.131818  0.294083   
gaussian_process_regressor       PESO_20_ahead1  57425.852302  0.629765   
                                 PESO_20_ahead2  67402.100538  1.295701   
                                 PESO_20_ahead3  80154.987483  1.030278   
                                 PESO_20_ahead4  26172.507946  0.344068   
                                 PESO_20_ahead5  48156.382919  0.696800   
                                 PESO_20_ahead6  24514.616664  0.354418   
                                 PESO_20_mean    50637.741309  0.725172   
gaussian_process_regressor_chain PESO_20_ahead1  27210.504499  0.502692   
                                 PESO_20_ahead2  21814.660739  0.394897   
                                 PESO_20_ahead3  25990.133398  0.319145   
                                 PESO_20_ahead4  28226.867808  0.377248   
                                 PESO_20_ahead5  33212.227261  0.490093   
                                 PESO_20_ahead6  20328.424718  0.310922   
                                 PESO_20_mean    26130.469737  0.399166   
gradient_boost                   PESO_20_ahead1  31309.502091  0.447852   
                                 PESO_20_ahead2  36412.297360  0.729944   
                                 PESO_20_ahead3  18783.724883  0.252194   
                                 PESO_20_ahead4  20634.390074  0.256843   
                                 PESO_20_ahead5  23913.436752  0.348881   
                                 PESO_20_ahead6  15870.560094  0.237582   
                                 PESO_20_mean    24487.318542  0.378883   
random_forest                    PESO_20_ahead1  23368.970026  0.454552   
                                 PESO_20_ahead2  24342.970139  0.390360   
                                 PESO_20_ahead3  21437.822394  0.307997   
                                 PESO_20_ahead4  22614.769075  0.323438   
                                 PESO_20_ahead5  19990.059085  0.299610   
                                 PESO_20_ahead6  17230.726888  0.249220   
                                 PESO_20_mean    21497.552934  0.337529   

                                                         rmse           mse  
decision_tree_regressor          PESO_20_ahead1  37827.967217  1.430955e+09  
                                 PESO_20_ahead2  28463.896565  8.101934e+08  
                                 PESO_20_ahead3  63331.931043  4.010933e+

In [301]:
results_p7

Model   decision_tree_regressor                                            \
Type                       Pred                                             
Horizon           PESO_7_ahead1 PESO_7_ahead2 PESO_7_ahead3 PESO_7_ahead4   
Ago/17             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
Sep/17             9.780713e+06     2429763.0  1.459708e+06  1.421997e+06   
Oct/17             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
Nov/17             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
Dic/17             5.635520e+05      934037.0  8.356420e+06  8.666680e+05   
Ene/18             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
Feb/18             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
Mar/18             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
Abr/18             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
May/18             1.724137e+06     1449678.0  1.448556e+06  1.284669e+06   
Jun/18             9.340370e+05     8356420.0  8.666680e+05  1.182016e+06   

Model                                                                          \
Type                                         Real                               
Horizon PESO_7_ahead5 PESO_7_ahead6 PESO_7_ahead1 PESO_7_ahead2 PESO_7_ahead3   
Ago/17   1.572633e+06  1.461600e+06      866668.0     1182016.0     1300326.0   
Sep/17   6.760690e+05  1.061285e+06     1182016.0     1300326.0     3265438.0   
Oct/17   1.572633e+06  1.461600e+06     1300326.0     3265438.0    10891638.0   
Nov/17   1.572633e+06  1.461600e+06     3265438.0    10891638.0     4426497.0   
Dic/17   1.182016e+06  1.300326e+06    10891638.0     4426497.0     3188769.0   
Ene/18   1.572633e+06  1.461600e+06     4426497.0     3188769.0     2974767.0   
Feb/18   1.572633e+06  1.461600e+06     3188769.0     2974767.0     4204833.0   
Mar/18   1.572633e+06  1.461600e+06     2974767.0     4204833.0     4927018.0   
Abr/18   1.572633e+06  1.461600e+06     4204833.0     4927018.0     9058866.0   
May/18   1.572633e+06  1.461600e+06     4927018.0     9058866.0     8036733.0   
Jun/18   1.300326e+06  3.265438e+06     9058866.0     8036733.0    13186744.0   

Model                      ...      random_forest                              \
Type                       ...               Pred                               
Horizon PESO_7_ahead4      ...      PESO_7_ahead3 PESO_7_ahead4 PESO_7_ahead5   
Ago/17      3265438.0      ...       2.974239e+06  2.165487e+06  2.426601e+06   
Sep/17     10891638.0      ...       2.883173e+06  2.496621e+06  2.568816e+06   
Oct/17      4426497.0      ...       2.845402e+06  2.539683e+06  2.349065e+06   
Nov/17      3188769.0      ...       2.977158e+06  2.487952e+06  2.481128e+06   
Dic/17      2974767.0      ...       3.086795e+06  2.601858e+06  2.469998e+06   
Ene/18      4204833.0      ...       3.058066e+06  2.711476e+06  2.338837e+06   
Feb/18      4927018.0      ...       3.464389e+06  2.054809e+06  2.193141e+06   
Mar/18      9058866.0      ...       3.013885e+06  1.961403e+06  1.761365e+06   
Abr/18      8036733.0      ...       3.164837e+06  1.897863e+06  1.730889e+06   
May/18     13186744.0      ...       3.144177e+06  1.898616e+06  1.565735e+06   
Jun/18       739023.0      ...       2.976845e+06  2.038646e+06  2.040782e+06   

Model                                                                          \
Type                           Real                                             
Horizon PESO_7_ahead6 PESO_7_ahead1 PESO_7_ahead2 PESO_7_ahead3 PESO_7_ahead4   
Ago/17   2.318915e+06      866668.0     1182016.0     1300326.0     3265438.0   
Sep/17   2.345645e+06     1182016.0     1300326.0     3265438.0    10891638.0   
Oct/17   2.005625e+06     1300326.0     3265438.0    10891638.0     4426497.0   
Nov/17   2.094912e+06     3265438.0    10891638.0     4426497.0     3188769.0   
Dic/17   2.101953e+06    10891638.0     4426497.0     3188769.0     2974767.0   
Ene

In [212]:
results_p7["adaboost_tree_regressor"]

Type             Pred                                                          \
Horizon PESO_7_ahead1 PESO_7_ahead2 PESO_7_ahead3 PESO_7_ahead4 PESO_7_ahead5   
Ago/17      1318255.0     1495317.0     1569340.0     3047421.0  2.767615e+06   
Sep/17      1495317.0     1569340.0     3047421.0     2487279.0  2.487279e+06   
Oct/17      2370005.0     2715935.0     2715935.0     2487279.0  2.767615e+06   
Nov/17      2767615.0     2556898.0     2487279.0     2715935.0  2.487279e+06   
Dic/17      2556898.0     2556898.0     2767615.0     2450521.0  2.450521e+06   
Ene/18      2487279.0     2767615.0     2487279.0     2429763.0  1.569340e+06   
Feb/18      2767615.0     2767615.0     2450521.0     1495317.0  1.300326e+06   
Mar/18      3047421.0     2450521.0     2240413.0     1300326.0  1.300326e+06   
Abr/18      2458521.0     2240413.0     1459708.0     1300326.0  1.325578e+06   
May/18      2370005.0     1569340.0     1310932.0     2240413.0  1.293670e+06   
Jun/18      1495317.0     2240413.0     1459708.0     1312952.0  1.452166e+06   

Type                           Real                                            \
Horizon PESO_7_ahead6 PESO_7_ahead1 PESO_7_ahead2 PESO_7_ahead3 PESO_7_ahead4   
Ago/17   3.663372e+06      866668.0     1182016.0     1300326.0     3265438.0   
Sep/17   3.265438e+06     1182016.0     1300326.0     3265438.0    10891638.0   
Oct/17   3.265438e+06     1300326.0     3265438.0    10891638.0     4426497.0   
Nov/17   2.370005e+06     3265438.0    10891638.0     4426497.0     3188769.0   
Dic/17   1.310932e+06    10891638.0     4426497.0     3188769.0     2974767.0   
Ene/18   1.182016e+06     4426497.0     3188769.0     2974767.0     4204833.0   
Feb/18   1.303861e+06     3188769.0     2974767.0     4204833.0     4927018.0   
Mar/18   1.300326e+06     2974767.0     4204833.0     4927018.0     9058866.0   
Abr/18   1.310932e+06     4204833.0     4927018.0     9058866.0     8036733.0   
May/18   1.325578e+06     4927018.0     9058866.0     8036733.0    13186744.0   
Jun/18   2.429763e+06     9058866.0     8036733.0    13186744.0      739023.0   

Type                                 
Horizon PESO_7_ahead5 PESO_7_ahead6  
Ago/17     10891638.0     4426497.0  
Sep/17      4426497.0     3188769.0  
Oct/17      3188769.0     2974767.0  
Nov/17      2974767.0     4204833.0  
Dic/17      4204833.0     4927018.0  
Ene/18      4927018.0     9058866.0  
Feb/18      9058866.0     8036733.0  
Mar/18      8036733.0    13186744.0  
Abr/18     13186744.0      739023.0  
May/18       739023.0     2108616.0  
Jun/18      2108616.0     2938267.0

In [210]:
results_p7["random_forest"]

Type             Pred                                                          \
Horizon PESO_7_ahead1 PESO_7_ahead2 PESO_7_ahead3 PESO_7_ahead4 PESO_7_ahead5   
Ago/17   2.871721e+06  2.323600e+06  2.154212e+06  2.887837e+06  2.390239e+06   
Sep/17   2.731094e+06  2.363147e+06  2.224438e+06  2.283120e+06  2.709601e+06   
Oct/17   3.369098e+06  2.379092e+06  2.121660e+06  2.696978e+06  2.552853e+06   
Nov/17   2.571847e+06  2.537592e+06  2.685582e+06  2.947895e+06  2.566605e+06   
Dic/17   2.439312e+06  2.480874e+06  3.248451e+06  2.870912e+06  2.249967e+06   
Ene/18   2.853320e+06  3.219084e+06  3.067899e+06  2.207741e+06  2.575393e+06   
Feb/18   2.689336e+06  3.514933e+06  2.766087e+06  2.357693e+06  1.979541e+06   
Mar/18   2.900830e+06  2.805964e+06  2.800471e+06  1.872804e+06  2.026133e+06   
Abr/18   2.786272e+06  3.220011e+06  2.772291e+06  2.051086e+06  2.263457e+06   
May/18   3.063699e+06  2.946198e+06  2.591194e+06  2.748274e+06  1.769189e+06   
Jun/18   2.660439e+06  3.109372e+06  2.301536e+06  1.855915e+06  2.316871e+06   

Type                           Real                                            \
Horizon PESO_7_ahead6 PESO_7_ahead1 PESO_7_ahead2 PESO_7_ahead3 PESO_7_ahead4   
Ago/17   4.467761e+06      866668.0     1182016.0     1300326.0     3265438.0   
Sep/17   4.216478e+06     1182016.0     1300326.0     3265438.0    10891638.0   
Oct/17   4.209863e+06     1300326.0     3265438.0    10891638.0     4426497.0   
Nov/17   2.428614e+06     3265438.0    10891638.0     4426497.0     3188769.0   
Dic/17   2.518275e+06    10891638.0     4426497.0     3188769.0     2974767.0   
Ene/18   2.315695e+06     4426497.0     3188769.0     2974767.0     4204833.0   
Feb/18   2.422002e+06     3188769.0     2974767.0     4204833.0     4927018.0   
Mar/18   2.384861e+06     2974767.0     4204833.0     4927018.0     9058866.0   
Abr/18   2.284025e+06     4204833.0     4927018.0     9058866.0     8036733.0   
May/18   2.036073e+06     4927018.0     9058866.0     8036733.0    13186744.0   
Jun/18   2.487561e+06     9058866.0     8036733.0    13186744.0      739023.0   

Type                                 
Horizon PESO_7_ahead5 PESO_7_ahead6  
Ago/17     10891638.0     4426497.0  
Sep/17      4426497.0     3188769.0  
Oct/17      3188769.0     2974767.0  
Nov/17      2974767.0     4204833.0  
Dic/17      4204833.0     4927018.0  
Ene/18      4927018.0     9058866.0  
Feb/18      9058866.0     8036733.0  
Mar/18      8036733.0    13186744.0  
Abr/18     13186744.0      739023.0  
May/18       739023.0     2108616.0  
Jun/18      2108616.0     2938267.0